In [1]:
import os
import torch
import torchvision as tv
import h5py
import numpy as np
import torch
import fastmri
import random
import sys
import torchvision
import pytorch_lightning as pl

sys.path.append('../../src/')
from helpers import calculate_metrics

from pathlib import Path
from torch import nn
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils import data
from torch.utils.tensorboard import SummaryWriter
from PIL import Image
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# try:
#     print(torch.cuda.get_device_name())
#     print(torch.cuda.get_device_properties(0))
# except:
#     print("No GPUs configured")
    
manualSeed = 42
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  42


# PyTorch Dataset

In [53]:
from fastmri.data.mri_data import fetch_dir

path_config = Path("../../fastmri_dirs.yaml")

dataroot = fetch_dir("knee_path", path_config)
test_path = Path("../../mnt/FastMRI/")
dataroot

PosixPath('/mri-reconstruction/mnt/FastMRI/knee')

# Check for truncated files 

In [76]:
import os

for i in dataroot.glob("**/*.h5"):
    try:
        hf = h5py.File(i, mode='r')
    except:
        print("Truncated: ", i)
#         os.remove("{}".format(i))

In [73]:
from fastmri.pl_modules import FastMriDataModule, UnetModule
from fastmri.data.transforms import UnetDataTransform
from fastmri.data.subsample import create_mask_for_mask_type

# singlecoil or multicoil
challenge = "singlecoil"
# MASK_TYPE is either random (for knee) or equispaced (for brain)
mask_type = "random"
center_fractions = [0.08]
accelerations = [4]

mask = create_mask_for_mask_type(
    mask_type, center_fractions, accelerations
)

train_transform = UnetDataTransform(challenge, mask_func=mask, use_seed=False)
val_transform = UnetDataTransform(challenge, mask_func=mask)
test_transform = UnetDataTransform(challenge)
    
data_module = FastMriDataModule(
    data_path=dataroot,
    challenge=challenge,
    train_transform=train_transform,
    val_transform=val_transform,
    test_transform=test_transform,
    test_path=test_path
)

# UNet Model

In [74]:
model = UnetModule(
    in_chans=1,  # number of input channels to U-Net
    out_chans=1,  # number of output chanenls to U-Net
    chans=32,  # number of top-level U-Net channels
    num_pool_layers=4,  # number of U-Net pooling layers
    drop_prob=0.0,  # dropout probability
    lr=0.001,  # RMSProp learning rate
    lr_step_size=40,  # epoch at which to decrease learning rate
    lr_gamma=0.1,  # extent to which to decrease learning rate
    weight_decay=0.0,  # weight decay regularization strength
)

# Train model

In [78]:
trainer = pl.Trainer()
trainer.fit(model, datamodule=data_module)

GPU available: False, used: False
INFO:lightning:GPU available: False, used: False
TPU available: None, using: 0 TPU cores
INFO:lightning:TPU available: None, using: 0 TPU cores

  | Name             | Type                 | Params
----------------------------------------------------------
0 | NMSE             | DistributedMetricSum | 0     
1 | SSIM             | DistributedMetricSum | 0     
2 | PSNR             | DistributedMetricSum | 0     
3 | ValLoss          | DistributedMetricSum | 0     
4 | TotExamples      | DistributedMetricSum | 0     
5 | TotSliceExamples | DistributedMetricSum | 0     
6 | unet             | Unet                 | 7.8 M 
----------------------------------------------------------
7.8 M     Trainable params
0         Non-trainable params
7.8 M     Total params
31.024    Total estimated model params size (MB)
INFO:lightning:
  | Name             | Type                 | Params
----------------------------------------------------------
0 | NMSE             

/opt/conda/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1